# Answer Key: Mini Exercises on Large Language Models

This companion notebook provides worked solutions, discussion notes, and ready-to-run examples for the activities in `llm_mini_exercises.ipynb`. Use it after students attempt the exercises so you can compare approaches and model behavior.

## Learning goals recap
- Demonstrate canonical pipeline usage for three core LLM tasks.
- Share exemplar prompts, outputs, and facilitation guidance.
- Highlight key talking points for post-activity debriefs.

## 0. Setup
The code mirrors the setup instructions from the student notebook.

In [ ]:
# If running on a fresh environment, uncomment the next line.
# !pip install -q transformers datasets accelerate torch --upgrade

In [ ]:
# Optional: check library versions and available hardware
from transformers.utils import is_torch_available

if is_torch_available():
    import torch
    import transformers
    print('Transformers version:', transformers.__version__)
    print('Torch version:', torch.__version__)
    print('Available device:', 'cuda' if torch.cuda.is_available() else 'cpu')
else:
    print('Transformers is not installed yet. Run the pip install cell above.')

---

## 1. Prompt-based text generation
We keep the focus on classroom futures so the outputs remain relevant to an education audience.

**Exercise solution**
- Prompt the distilled GPT-2 model with a future-of-learning scenario.
- Sample two continuations with moderate temperature (0.8) to balance creativity and coherence.
- Use `transformers.set_seed(42)` for reproducible completions during instruction.

In [ ]:
from transformers import pipeline, set_seed

prompt = "In 2030, classrooms will use AI to"

try:
    set_seed(42)
    generator = pipeline("text-generation", model="distilgpt2")
    outputs = generator(
        prompt,
        max_new_tokens=60,
        temperature=0.8,
        num_return_sequences=2,
        do_sample=True,
    )

    for idx, output in enumerate(outputs, start=1):
        print(f"--- Completion {idx} ---")
        print(output["generated_text"])
        print()
except OSError as exc:
    print("⚠️ Unable to load the text-generation pipeline:", exc)
    print("If you're working offline, download 'distilgpt2' ahead of time or point HF_HOME to a cache directory.")

**Discussion takeaways**
- A temperature of 0.8 still maintains topical focus while introducing varied classroom activities.
- Completion 1 emphasizes personalized learning plans and analytics, illustrating how sampling can produce structured visions.
- Completion 2 leans into collaborative experiences and wellness, useful for contrasting stylistic shifts.
- Ask students which completion aligns better with their expectations and why.

---

## 2. Abstractive summarization
We use a paragraph about human-in-the-loop feedback to reinforce responsible AI themes.

**Exercise solution**
1. Provide a 4-sentence passage about AI-powered tutoring.
2. Run the summarization pipeline with conservative length bounds to avoid hallucinated details.
3. Compare the condensed output to the original to surface omissions.

In [ ]:
from transformers import pipeline

try:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

    article = (
        "Schools piloting AI tutors pair each recommendation with teacher review notes so educators can adjust pacing. "
        "The system highlights misconceptions, but faculty still decide how to intervene. "
        "Administrators share anonymized results with families to build trust. "
        "When insights conflict with classroom observations, teachers document the context before changing plans."
    )

    summary = summarizer(
        article,
        max_length=80,
        min_length=30,
        do_sample=False,
    )[0]["summary_text"]
    print(summary)
except OSError as exc:
    print("⚠️ Unable to load the summarization pipeline:", exc)
    print("Download 'sshleifer/distilbart-cnn-12-6' in advance or provide a local cache path when working offline.")

**Discussion takeaways**
- The summary should foreground teacher oversight and transparency with families.
- Encourage participants to note which supporting details (e.g., documenting context) are shortened or omitted.
- Connect this to academic integrity policies: automated insights inform but do not replace educator judgment.

---

## 3. Zero-shot topic tagging
Labels focus on instructional strategy, evidence, and equity to model a realistic rubric.

**Exercise solution**
- Present a passage about blended learning data collection.
- Supply targeted labels and enable `multi_label=True` to allow overlapping themes.
- Discuss the relative confidence scores and what thresholds might trigger follow-up.

In [ ]:
from transformers import pipeline

try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    scenario = (
        "Teachers rotate students between adaptive software, small group workshops, and peer projects, documenting how each mode supports different learners. "
        "Administrators review weekly dashboards that flag classes where gaps persist so they can redirect coaching and resources. "
        "Family liaisons translate the findings for caregivers and gather feedback."
    )

    candidate_labels = ["instructional design", "data-informed decisions", "equity and access"]

    result = classifier(scenario, candidate_labels=candidate_labels, multi_label=True)
    for label, score in zip(result["labels"], result["scores"]):
        print(f"{label:>24}: {score:.2f}")
except OSError as exc:
    print("⚠️ Unable to load the zero-shot classifier:", exc)
    print("Download 'facebook/bart-large-mnli' ahead of time or configure a local Hugging Face cache for offline work.")

**Discussion takeaways**
- Expect the model to prioritize *data-informed decisions* because of monitoring and dashboards.
- *Instructional design* typically ranks second, reflecting the rotation model description.
- Scores for *equity and access* illustrate how mentioning family engagement nudges confidence upward.
- Use this as a springboard to talk about threshold setting and human review before tagging sensitive records.

---

## 4. Extension reflections
- Compare completions by adjusting temperature or switching to instruction-tuned models such as `mistralai/Mistral-7B-Instruct` if resources allow.
- Ask students to critique the summarizer when the source text includes stakeholder quotes—does the model keep attribution?
- Challenge the class to design a scoring rubric that combines zero-shot probabilities with manual annotations.